<a href="https://colab.research.google.com/github/van-dang/MRI-Cloud/blob/master/StrongPeriodicBC_TwoComp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#About the solver

In [0]:
# This demo solves the transformed Bloch-Torrey equation applied to diffusion MRI 
# using the standard finite element method coupled with the theta-method 
# for the space discretization.

# This demo allows for a two-compartment domain with periodic boundaries and the 
# spins can enter and exit the computational domain freely by using the periodic 
# boundary conditions

# Copyright (C) 2017 Van-Dang Nguyen

# This file is part of DOLFIN.

# DOLFIN is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# DOLFIN is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU Lesser General Public License for more details.

# You should have received a copy of the GNU Lesser General Public License
# along with DOLFIN. If not, see <http://www.gnu.org/licenses/>.

# First added:  2017-10-10
# Last changed: 2019-04-10

# Setting a working environment

In [0]:
from google.colab import files

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
    
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *
    

import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time

import os, sys, shutil

clear_output(); # Plotting setup

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)


!rm -rf * # clean up all files

# dolfin setting
q_degree = 3
dx = dx(metadata={'quadrature_degree': q_degree})

# Useful commands
# Remove an empty folder      : os.rmdir("my_results")
# Remove a folder with files  : shutil.rmtree("results")
# Make a folder               : os.mkdir("my_results")
# Runtime/Change_runtime_type/Python3

E: Package 'python-software-properties' has no installation candidate
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,609 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/fenics-packages/fenics/ubuntu bionic InRelease [15.4 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [60.4 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease

In [0]:
delta, Delta = 50000, 50000

gnorm = sqrt(bvalue)/sqrt(delta*delta*(Delta-delta/3.0));
print sys.argv
print "Nsteps: ", Nsteps, "bvalue: ",bvalue


dolfin_version = dolfin.dolfin_version()
print 'dolfin version:', dolfin_version

def SubMeshSave(ur, ui, file_ur, file_ui, mesh, n, stepcounter, dolfin_version):
  if dolfin_version=='1.6.0':
    V = FunctionSpace(mesh, "CG", porder)
  else:
    # For FEniCS 2016, 2017
    Ve = FiniteElement("CG", mesh.ufl_cell(), porder)
    V = FunctionSpace(mesh, Ve)
  if stepcounter % n == 0:
    ur_p = project(ur, V);
    ui_p = project(ui, V);
    ur_p.rename("Real", "label");
    ui_p.rename("Imag", "label");
    file_ur << ur_p;
    file_ui << ui_p;

def ieval(u,omega, phase):
  if omega==1:
    return u('+')*phase('+') + u('-')*phase('-');
  if omega==0:
    return u('+')*(1.-phase('+')) + u('-')*(1.-phase('-'))                              

#####################################################################################################################
#####################################################################################################################
# GEOMETRY SETTINGS
xmin, ymin, zmin, xmax, ymax, zmax = -10, -10, -10, 10, 10, 10


mesh=Mesh('mesh2comp_R9_nonperiodic.xml');

hmin = mesh.hmin();
center=[];
center.append([-4,0,0]);
center.append([16,0,0]);
R0=9;

print center[0][0], center[0][1], center[0][2]
print center[1][0], center[1][1], center[1][2]

class Exterior(SubDomain):
    def inside (self,x,on_boundary):
        d0 = sqrt((x[0]-center[0][0])*(x[0]-center[0][0])+(x[1]-center[0][1])*(x[1]-center[0][1])+(x[2]-center[0][2])*(x[2]-center[0][2]));
        d1 = sqrt((x[0]-center[1][0])*(x[0]-center[1][0])+(x[1]-center[1][1])*(x[1]-center[1][1])+(x[2]-center[1][2])*(x[2]-center[1][2]));
        return (d0>=R0-hmin/4.0 and d1>=R0-hmin/4.0);

class Interior(SubDomain):
    def inside (self,x,on_boundary):
        exterior = Exterior()
        return not(exterior.inside(x, True))

# Mark exterior domain to 1, interior domain to 0
exterior = Exterior();
interior = Interior();
cellmarker = CellFunction("size_t", mesh)
cellmarker.set_all(0)
exterior.mark(cellmarker,1)

# plot(cellmarker); interactive(); stop
# stop;

mesh0 = SubMesh(mesh, cellmarker, 0)
mesh1 = SubMesh(mesh, cellmarker, 1)
V_DG = FunctionSpace(mesh, 'DG', 0)
dofmap_DG = V_DG.dofmap()
phase = Function(V_DG)
vol = CellVolume(mesh)
h = 0.5*CellSize(mesh);

for cell in cells(mesh):
    phase.vector()[dofmap_DG.cell_dofs(cell.index())] = cellmarker[cell.index()];

# plot(phase); interactive(); stop
mesh_file = File("mesh.xml")
mesh_file << mesh
phase_file = File("Phi.xml")
phase_file << phase

phase_file = File("Phi.pvd")
phase_file << phase

# stop

########## END OF GEOMETRY SETTINGS
#####################################################################################################################

# parameters
#####################################################################################################################
#####################################################################################################################
porder = 1;

# delta, Delta = 50000, 50000
t, T = 0, Delta+delta;
K0, K1 = 3e-3, 3e-3;
K = 3e-3;
g0, g1, g2 = 1.0, 0.0, 0.0

kappa = 1e-5; # permeability
# kappa = 1.0; # permeability

nskip = 2;

# bvalue = 1000;
# gnorm = sqrt(bvalue)/sqrt(delta*delta*(Delta-delta/3.0));
#####################################################################################################################
#####################################################################################################################

#####################################################################################################################
#####################################################################################################################
# FUNCTION SPACES
if dolfin_version=='1.6.0':
    V = FunctionSpace(mesh , "CG", 1);
    W = MixedFunctionSpace([V, V, V, V])
    VV = MixedFunctionSpace([V, V])
else:
    # For FEniCS 2016, 2017
    Ve = FiniteElement("CG", mesh.ufl_cell(), 1)
    TH = MixedElement([Ve,Ve,Ve,Ve])
    VVe = MixedElement([Ve,Ve])
    V = FunctionSpace(mesh,Ve);
    VV = FunctionSpace(mesh,VVe);
    W = FunctionSpace(mesh, TH)

v = TestFunction(W)
v0r, v0i, v1r, v1i = v[0], v[1], v[2], v[3]

w = TrialFunction(W);
u0r, u0i, u1r, u1i = w[0], w[1], w[2], w[3]
#####################################################################################################################
#####################################################################################################################
# Initial conditions
# one = Function(V);
Dirac_Delta = Expression("x[0]*x[0]+x[1]*x[1]+x[2]*x[2]<eps",eps=1e6, domain=mesh, degree=2);
Dirac_Delta = interpolate(Dirac_Delta, V);
# plot(Dirac_Delta); interactive(); stop;
# one.vector()[:] = 1;

'''
u_0 = Function(W);
assign(u_0.sub(0), Dirac_Delta)
assign(u_0.sub(2), Dirac_Delta)
u0r_0, u0i_0, u1r_0, u1i_0 = u_0[0], u_0[1], u_0[2], u_0[3]
'''

u = Function(W);
assign(u.sub(0), Dirac_Delta)
assign(u.sub(2), Dirac_Delta)

#####################################################################################################################
#####################################################################################################################

g = Expression(("g0","g1","g2"), g0=g0, g1=g1, g2=g2,domain=mesh,degree=2);

#####################################################################################################################
#####################################################################################################################


GX=Expression("x[0]*g0+x[1]*g1+x[2]*g2", g0=g0, g1=g1, g2=g2,domain=mesh,degree=3);

#################################                                                                                                                                                         
## output files                                                                                                                                                                           
file_u0r = File("results/u0r.pvd")
file_u0i = File("results/u0i.pvd")
file_u1r = File("results/u1r.pvd")
file_u1i = File("results/u1i.pvd")
#################################                                                                                                                                                         

def FT(t, delta, Delta):
    ft1 = 1.0*(t>=0 and t<delta)
    ft2 = -1.0*(t>=Delta and t<=Delta+delta);
    return ft1 + ft2;

def iFT(t, delta, Delta): # integrate ft                                                                                                                                                  
    ft1 = t*(t>=0 and t<delta)
    ft2 = delta*(t>=delta and t<Delta)
    ft3 = (delta - t + Delta)*(t>=Delta and t<=Delta+delta)
    return ft1 + ft2 + ft3;

stepcounter = 0;

def FuncF(ft, gnorm, GX, ur, ui, vr, vi, K):
    Fr = ft*gnorm*GX*ui*vr - K*inner(grad(ur), grad(vr))
    Fi = - ft*gnorm*GX*ur*vi - K*inner(grad(ui), grad(vi))
    return Fr + Fi

def icondition(kappa, u0rm, u1rm, v0r, v1r, u0im, u1im, v0i, v1i):
    F_bcr = kappa*(u0rm-u1rm)*(v0r-v1r)
    F_bci = kappa*(u0im-u1im)*(v0i-v1i)
    return F_bcr + F_bci


##########
##########
class PseudoPeriodic(Expression):
    def set_values(self, ur, ui, xmin, ymin, xmax, ymax, ift, dir):
        self.ur, self.ui                               = ur, ui
        self.xmin, self.ymin, self.xmax, self.ymax     = xmin, ymin, xmax, ymax 
        self.dir = dir
    def eval(self, value, x):
        tol = 1E-7
        is_eval = False;
        # exp(i*theta)= cos(theta)+i*sin(theta); 
        xln, yln = 0, 0
        temp_ur, temp_ui = 0, 0
        if (self.dir == 0): # x-direction
            if abs(x[0]-self.xmin) <= tol:
                xln, yln = self.xmax-x[0], 0;
                temp_ur = self.ur([self.xmax, x[1], x[2]]);
                temp_ui = self.ui([self.xmax, x[1], x[2]]);
            if abs(x[0]-self.xmax) <= tol:
                xln, yln = self.xmin-x[0], 0;
                temp_ur = self.ur([self.xmin, x[1], x[2]]);
                temp_ui = self.ui([self.xmin, x[1], x[2]]);

        if (self.dir == 1): # y-direction
            if abs(x[1]-self.ymin) <= tol:
                xln, yln = 0, self.ymax-x[1];
                temp_ur = self.ur([x[0], self.ymax]);
                temp_ui = self.ui([x[0], self.ymax]);

            if abs(x[1]-self.ymax) <= tol:
                xln, yln = 0, self.ymin-x[1];
                temp_ur = self.ur([x[0], self.ymin]);
                temp_ui = self.ui([x[0], self.ymin]);

        theta_ln = gnorm*(g0*xln + g1*yln)*ift;
        value[0] = temp_ur*cos(theta_ln)-temp_ui*sin(theta_ln); # for Real part
        value[1] = temp_ur*sin(theta_ln)+temp_ui*cos(theta_ln); # for Imag part

    def value_shape(self):
        return (2,)


class PseudoPeriodic0(Expression):
    def set_values(self, u, xmin, ymin, xmax, ymax, ift, dir):
        self.u                                         = u
        self.xmin, self.ymin, self.xmax, self.ymax     = xmin, ymin, xmax, ymax 
        self.dir = dir
    def eval(self, value, x):
        tol = 1E-7
        is_eval = False;
        xln, yln, zln = 0, 0, 0
        temp_u0r, temp_u0i, temp_u1r, temp_u1i = 0, 0, 0, 0
        if (self.dir == 0): # x-direction
            if abs(x[0]-self.xmin) <= tol:
                xln, yln, zln = self.xmax-x[0], 0, 0;
                temp_u0r, temp_u0i, temp_u1r, temp_u1i = self.u([self.xmax, x[1], x[2]]);

            if abs(x[0]-self.xmax) <= tol:
                xln, yln, zln = self.xmin-x[0], 0, 0;
                temp_u0r, temp_u0i, temp_u1r, temp_u1i = self.u([self.xmin, x[1], x[2]]);

        # exp(i*theta)= cos(theta)+i*sin(theta); 
        theta_ln = gnorm*(g0*xln + g1*yln + g2*zln)*ift;
        value[0] = temp_u0r*cos(theta_ln)-temp_u0i*sin(theta_ln); # for Real part
        value[1] = temp_u0r*sin(theta_ln)+temp_u0i*cos(theta_ln); # for Imag part

        value[2] = temp_u1r*cos(theta_ln)-temp_u1i*sin(theta_ln); # for Real part
        value[3] = temp_u1r*sin(theta_ln)+temp_u1i*cos(theta_ln); # for Imag part

    def value_shape(self):
        return (4,)

# pperiodic = PseudoPeriodic(degree=1)
pperiodic0 = PseudoPeriodic0(degree=1)

#########################################################
################ Functions
def ThetaMethod_L(ft, gnorm, GX, u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, u0r_0, u0i_0, u1r_0, u1i_0,k, kappa, K, theta, phase):
    L0 = (u0r_0/k*v0r + u0i_0/k*v0i+(1-theta)*FuncF(ft, gnorm, GX, u0r_0, u0i_0, v0r, v0i, K))*(1-phase)*dx
    L1 = (u1r_0/k*v1r +u1i_0/k*v1i+(1-theta)*FuncF(ft, gnorm, GX, u1r_0, u1i_0, v1r, v1i, K))*phase*dx
    L_bc = avg((1-theta)*icondition(kappa, u0r_0, u1r_0, v0r, v1r, u0i_0, u1i_0, v0i, v1i))*abs(jump(phase))*dS;
    return L0+L1-L_bc


def ThetaMethod_a(ft, gnorm, GX, u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, u0r_0, u0i_0, u1r_0, u1i_0,k,kappa, K, theta, phase):
    a0 = (u0r/k*v0r   + u0i/k*v0i  -theta*FuncF(ft, gnorm, GX, u0r  , u0i  , v0r, v0i, K))*(1-phase)*dx
    a1 = (u1r/k*v1r   + u1i/k*v1i  -theta*FuncF(ft, gnorm, GX, u1r  , u1i  , v1r, v1i, K))*phase*dx
    a_bc = avg(  (theta*icondition(kappa, u0r  , u1r  , v0r, v1r, u0i  , u1i  , v0i, v1i)))*abs(jump(phase))*dS;
    return a0+a1+a_bc


def NoTimeMatrices(u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, K, GX, kappa, theta, phase, kappa_e):
    m0 = (u0r*v0r   + u0i*v0i)*(1-phase)*dx
    m1 = (u1r*v1r   + u1i*v1i)*phase*dx
    M = assemble(m0+m1);
        
    j0 = -GX*(u0i*v0r   - u0r*v0i)*(1-phase)*dx
    j1 = -GX*(u1i*v1r   - u1r*v1i)*phase*dx
    J = assemble(j0+j1);    
    s0 = K*( inner(grad(u0r), grad(v0r)) + inner(grad(u0i), grad(v0i)) )*(1-phase)*dx
    s1 = K*( inner(grad(u1r), grad(v1r)) + inner(grad(u1i), grad(v1i)) )*phase*dx
    S = assemble(s0+s1)

    im = avg(  icondition(kappa, u0r  , u1r  , v0r, v1r, u0i  , u1i  , v0i, v1i) )*abs(jump(phase))*dS;
    I = assemble(im)
    
    ii = kappa_e*(u1r*v1r   + u1i*v1i)*phase*ds + kappa_e*(u0r*v0r   + u0i*v0i)*(1-phase)*ds;

    II = assemble(ii)

    M.ident_zeros();
    
    return M, J, S, I, II

def ThetaMethod_A(ft, gnorm, theta, k, M, J, S, I):
    return 1./k*M + ft*gnorm*theta*J + theta*S + theta*I

# Theta method

stepcounter = 0;
theta = 0.5;
# Nsteps = 100
k = T/Nsteps;
print "dt: ", k
print "hmin: ",hmin

kappa_e = K/h*Expression("(x[0]<xmin+eps || x[0]>xmax-eps)", xmin=xmin, xmax=xmax, eps=1e-10, domain=mesh, degree=3);

# plot(project(kappa_e,V)); interactive(); stop;

M, J, S, I, II = NoTimeMatrices(u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, K, GX, kappa, theta, phase, kappa_e);

dir = 0

# u = Function(W)

while t < T: # Time-stepping loop
    if stepcounter % nskip == 0:
        print 't=%f '%t, 'gnorm: %f'%gnorm;

    ft = FT(t, delta, Delta);
    ift = iFT(t, delta, Delta);

    pperiodic0.set_values(u, xmin, ymin, xmax, ymax, ift, dir);
    u0r_bc, u0i_bc, u1r_bc, u1i_bc = pperiodic0[0], pperiodic0[1], pperiodic0[2], pperiodic0[3]
    
    L = ThetaMethod_L(ft, gnorm, GX, u0r, u0i, v0r, v0i, u1r, u1i, v1r, v1i, u[0], u[1], u[2], u[3],k, kappa, K, theta, phase);
    
    A = ThetaMethod_A(ft, gnorm, theta, k, M, J, S, I);
    A += II;
    L_bc =  kappa_e*(u1r_bc*v1r   + u1i_bc*v1i)*    phase*ds; # u1r_0, u1i_0
    L_bc += kappa_e*(u0r_bc*v0r   + u0i_bc*v0i)*(1-phase)*ds; # u0r_0, u0i_0
    L += L_bc;
    b = assemble(L);
 
    solve(A,u.vector(),b, "gmres", "ilu");
    
    # u0r_0, u0i_0, u1r_0, u1i_0 = split(u)
    
    # SubMeshSave(u0r_0, u0i_0, file_u0r, file_u0i, mesh0, nskip, stepcounter, dolfin_version);
    # SubMeshSave(u1r_0, u1i_0, file_u1r, file_u1i, mesh1, nskip, stepcounter, dolfin_version);
    
    t += k;
    stepcounter += 1;

# plot(u1r_0); interactive();

signal = assemble((phase*u[2]+(1-phase)*u[0])*dx)/assemble(Dirac_Delta*dx);
print('b:',bvalue, 'Signal: ', signal)